# EJERCICIO CLASE FINAL GRUPO 4

## INTEGRANTES:
- Carlos Chicaiza
- Emilio Mayorga
- Jessica Llumiguano
- Juan Vizuete


## Preprocesamiento

In [1]:
%pip install psycopg2

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for psycopg2: filename=psycopg2-2.9.10-cp312-cp312-linux_x86_64.whl size=635593 sha256=6e7b1912c8cc8e5dedc3fa05acdd89959fdeffaa69f983ddd99deaec5075dd03
  Stored in directory: /home/codespace/.cache/pip/wheels/ac/bb/ce/afa589c50b6004d3a06fc691e71bd09c9bd5f01e5921e5329b
Successfully built psycopg2
Note: you may need to restart the kernel to use updated packages.


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# base de datos
import psycopg2

### Lectura de los datos

In [ ]:
with open('./data/cybersecurity_intrusion_data.csv', 'r') as data:
    cabecera = data.readline()
    print(cabecera)

session_id,network_packet_size,protocol_type,login_attempts,session_duration,encryption_used,ip_reputation_score,failed_logins,browser_type,unusual_time_access,attack_detected



In [4]:
df = pd.read_csv('./data/cybersecurity_intrusion_data.csv')
print(df.shape)

(9537, 11)


### Conexión a la base de datos

In [6]:
try:
    connection = psycopg2.connect(
        user="postgres", 
        password="postgres",
        host="localhost",
        port="5432"
    )
    connection.autocommit = True
    cursor = connection.cursor()
    cursor.execute("SELECT datname FROM pg_database WHERE datname='cybersecurity_intrusion';")
    if not cursor.fetchone():
        cursor.execute("CREATE DATABASE cybersecurity_intrusion;")
        print("Base creada")
    else:
        print("Si existe")
    
    cursor.close()  #Cerrada conexion al base
    connection.close() #Cerrada conexion al servidor
        
except Exception as e:
        print(f"Error al crear la base de datos: {e}")
    

Error al crear la base de datos: could not connect to server: Connection refused
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?

